# Optional: downloading training texts

In [ ]:
!pip install dtrx

In [ ]:
!lang="vi"; wget -c https://dumps.wikimedia.org/${lang}wiki/latest/${lang}wiki-latest-pages-articles.xml.bz2 & wget https://object.pouta.csc.fi/OPUS-WikiMatrix/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-wikimedia/v20210402/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-CCAligned/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-XLEnt/v1.1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-JW300/v1b/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-TED2020/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-QED/v2.0a/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-GNOME/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-bible-uedin/v1/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-ParaCrawl/v8/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-TildeMODEL/v2018/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-DGT/v2019/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-EUbookshop/v2/mono/${lang}.txt.gz & wget https://object.pouta.csc.fi/OPUS-SETIMES/v2/mono/${lang}.txt.gz

In [ ]:
!dtrx *.txt.gz.* & dtrx *.txt.gz & dtrx *.bz2

In [ ]:
!cat *.txt > dataset.txt

# Downloading and cleaning Wikipedia

In [ ]:
!nohup curl -L -O https://dumps.wikimedia.org/ltwiki/latest/ltwiki-latest-pages-articles.xml.bz2 &

In [ ]:
!pip install wikiextractor


In [ ]:
!python3 -m wikiextractor.WikiExtractor -o wiki-lt/ --no-templates --processes 8 ltwiki-latest-pages-articles.xml.bz2

In [ ]:
!du; ls -s

In [ ]:
!cat wiki*/*/* > final_untokenised.txt

In [ ]:
!cat wiki*/AA/wiki_*0* > tokentrain_final.txt

# Training the tokeniser

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install youtokentome & pip install tokenizers

In [ ]:
import youtokentome as yttm

In [ ]:
yttm.BPE.train('/content/tokentrain_final.txt', 'model', 50000, 0.9999, n_threads=-1, pad_id=0, unk_id=1, bos_id=2, eos_id=3)

In [ ]:
!yttm vocab --model model > vocab.txt

In [ ]:
!sed 's/^ *[0-9]\+.//g' /content/vocab.txt > /content/cleanvocab.txt

In [ ]:
text_file = open("/content/cleanvocab.txt", "r")
lines = text_file.read().split('\n')

In [ ]:
lines = [w.replace('▁', ' ') for w in lines]
print(lines)

# Encoding texts and making a tokeniser.json

In [ ]:
initial = '{"version":"1.0","truncation":null,"padding":null,"added_tokens":[{"id":50256,"special":true,"content":"<|endoftext|>","single_word":false,"lstrip":false,"rstrip":false,"normalized":true}],"normalizer":null,"pre_tokenizer":{"type":"ByteLevel","add_prefix_space":false,"trim_offsets":true},"post_processor":{"type":"ByteLevel","add_prefix_space":true,"trim_offsets":false},"decoder":{"type":"ByteLevel","add_prefix_space":true,"trim_offsets":true},"model":{"dropout":null,"unk_token":null,"continuing_subword_prefix":"","end_of_word_suffix":"","fuse_unk":false,"vocab":{'
i = 0
for token in lines:
    if '\\' in token:
       token = token.replace('\\', '\\\\')
    if '"' in token:
       token = token.replace('"', '\\"')
    initial = initial + '"' + token + '":' + str(i) + ','
    i = i + 1
initial = initial[:-1]
initial = initial + '},"merges":[]}}'
print(initial)

In [ ]:
with open("aitextgen.tokeniser.json", "w") as text_file:
    text_file.write(initial)

In [ ]:
!yttm encode --model /content/model --n_threads 8 --dropout_prob 0.25 --output_type id < /content/final_untokenised.txt > final.txt

# If the above doesn't work


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())
from tokenizers import ByteLevelBPETokenizer
# initialize
tokenizer = ByteLevelBPETokenizer()
# and train
tokenizer.train(files='/content/your_dataset.txt', vocab_size=17000, min_frequency=0, special_tokens=lines)

In [ ]:
tokenizer.save("tokenizer.json")

# Saving as npy array

There may be a more efficient way of doing this. I tried using linecache to read specific lines and then multithreaded it but then np.save broke, I'm not sure why.

In [75]:
data_array = np.array([], dtype=np.uint16)

In [76]:
np.save('data.npy', data_array)

In [ ]:
from pathlib import Path
import numpy as np
import os
import re
file = open("translations_tokenised_new.txt")
i = 0
while 1:
    i = i + 50000000
    string_data = file.read(50000000)
    if not string_data:
        break
    list_data = re.split(' |\n',string_data)
    list_data = list(filter(None, list_data))
    arr = np.array(list_data, dtype=np.uint16)
    f_handle = open('data.npy', 'ab')
    np.save(f_handle, arr)
    f_handle.close()
    print('Line ' + str(i) + ' done!')
arr = []
string_data = ""
list_data = ""


In [ ]:
!gzip data.npy

In [ ]:
!cp data.npy.gz /content/drive/MyDrive/